### Imports and definitions

In [1]:
# Import libraries
from parse_and_plot_log import *
import matplotlib.pyplot as plt
%matplotlib widget

# Obs: the widget magic assumes you have installed NodeJS and 
# @jupyter-widgets/jupyterlab-manager jupyter extension
# 
# To do that, run commands:
# > conda install nodejs
# > jupyter labextension install @jupyter-widgets/jupyterlab-manager
# Then restart jupyter notebook to correctly display the plot

In [2]:
log_parsing('log/', 'maxigauge')

,CH1,CH2,CH3,CH4,CH5,CH6
2021-01-26 10:47:55.000,0.000000,0.00,0.0,0.0,0.00,0.00
2021-01-26 10:48:56.000,0.000000,0.00,0.0,0.0,0.00,0.00
2021-01-26 10:56:08.500,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-26 11:00:15.250,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-26 11:02:48.625,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-02-08 11:56:21.000,0.000005,0.03,438.0,461.0,8.19,1.53
2021-02-08 11:57:22.000,0.000005,0.03,438.0,461.0,8.22,1.53
2021-02-08 11:58:24.000,0.000005,0.03,447.0,461.0,8.26,1.53
2021-02-08 11:59:26.000,0.000005,0.03,439.0,461.0,8.30,1.53


In [3]:
plot_data('log/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Textarea(value='01-01-00 00:00:00', description='Starting time', style=DescriptionStyle(…